In [27]:
!pip install yfinance==0.1.67
!mamba install bs4==4.10.0 -y
!pip install nbformat==4.2.0

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.37
    Uninstalling yfinance-0.2.37:
      Successfully uninstalled yfinance-0.2.37
zsh:1: command not found: mamba
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.9.2
    Uninstalling nbformat-5.9.2:
      Successfully uninstalled nbformat-5.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.9.0 requires nbformat>=5.1, but you have nbformat 4.2.0 which is incompatible.
jupyter-server 2.12.5 requires nbformat>=5.3.0, but you have nbformat 4.2.0 which is incompatible.
nbconvert 7.14.2 requires nbformat>=5.7, but you have nbformat 4.2.0 which is incompatible.


In [28]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [29]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [30]:
import yfinance as yf
tesla = yf.Ticker('TSLA')

In [31]:
tesla_data = tesla.history(period="max")

In [32]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [71]:
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Step 1: Fetch the revenue data
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# Iterate over tables and find the one with "Tesla Quarterly Revenue"
for table in soup.find_all('table'):
    if table.find('th').getText().startswith("Tesla Quarterly Revenue"):
        for row in table.find("tbody").find_all("tr"):
            col = row.find_all("td")
            if len(col) != 2: continue
            Date = col[0].text
            Revenue = col[1].text.replace("$", "").replace(",", "")
            
            # Create a new row as a DataFrame
            new_row = pd.DataFrame({"Date": [Date], "Revenue": [Revenue]})
            
            # Concatenate the new row with the existing DataFrame
            tesla_revenue = pd.concat([tesla_revenue, new_row], ignore_index=True)

# Clean the revenue data
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',', '').str.replace('$', '')
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

# Step 2: Fetch stock data using yfinance
tesla_stock = yf.download("TSLA", start="2010-01-01", end="2025-01-01")

# Step 3: Merge stock data and revenue data on Date
tesla_revenue['Date'] = pd.to_datetime(tesla_revenue['Date'])
tesla_revenue.set_index('Date', inplace=True)

tesla_stock.reset_index(inplace=True)
tesla_combined = pd.merge(tesla_stock[['Date', 'Close']], tesla_revenue, left_on='Date', right_index=True, how='inner')

# Step 4: Create the Plotly figure for the dashboard
fig = go.Figure()

# Add Stock Price trace
fig.add_trace(go.Scatter(x=tesla_combined["Date"], y=tesla_combined["Close"], mode='lines', name='Tesla Stock Price'))

# Add Revenue trace
fig.add_trace(go.Scatter(x=tesla_combined["Date"], y=tesla_combined["Revenue"], mode='lines', name='Tesla Revenue'))

# Update layout for the figure
fig.update_layout(title="Tesla Stock and Revenue Dashboard", xaxis_title="Date", yaxis_title="Value")

# Step 5: Create Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Tesla Stock and Revenue Dashboard"),
    dcc.Graph(
        id='tesla-dashboard',
        figure=fig
    )
])

if __name__ == "__main__":
    app.run_server(debug=True)


[*********************100%%**********************]  1 of 1 completed
